# W&B → Cloud Deploy Lab — Step-by-step Notebook

**Purpose:** Teach students how to train a simple model, log it to Weights & Biases (W&B) as an artifact, create a minimal inference service (FastAPI) which downloads the model artifact at startup, containerize the service, and deploy it to Google Cloud Run. The notebook is written so the training + artifact steps can be executed in Colab or a Jupyter server. Docker & cloud deploy steps are runnable on local Ubuntu lab machines.

**What you will produce:**

- A logged W&B model artifact (`iris-rf`) in project `classroom-deploy`.
- A FastAPI inference app that downloads the artifact and serves `/predict`.
- A Dockerfile to containerize the app.
- Cloud Run deployment instructions (or substitute AWS/Azure as desired).

---


## Prerequisites (before starting)

- **W&B account & API key**: Make sure students have W&B accounts (academic) and their `WANDB_API_KEY`.
- **Python**: Colab or Python 3.8+ environment.
- **For Docker & deploy**: Local Ubuntu machines with Docker and Google Cloud SDK installed.
- **Optional**: GitHub account (for CI/CD exercise).

**Notes:**
- Colab: You can run the training and artifact steps in Colab, but building Docker images and deploying with `gcloud` must be done on local machines.



## 1) Install Python dependencies

Run this cell in Colab or your local notebook to install the required packages for the lab (training, logging, and local testing).

In [ ]:
print('Using UV to install dependencies...')
!uv add wandb scikit-learn joblib numpy dotenv
print('Installed dependencies: wandb, scikit-learn, joblib, numpy, dotenv')

Using UV to install dependencies...
Resolved 57 packages in 0.91ms
Audited 52 packages in 0.02ms
Installed dependencies: wandb, scikit-learn, joblib, numpy


### 1.1 Set your W&B API key

Using Dotenv to load the api key here

In [2]:
import dotenv
import os

dotenv.load_dotenv()
WANDB_API_KEY = os.getenv('WANDB_API_KEY')
print(f'W&B API Key loaded: {WANDB_API_KEY is not None}')

W&B API Key loaded: True


## 2) Login to W&B from the notebook

This will attempt to use `WANDB_API_KEY` from the environment. If not set, `wandb.login()` may prompt you interactively (Colab).

In [4]:
import wandb
try:
    wandb.login()
    print('wandb.login() successful (or WANDB_API_KEY used).')
except Exception as e:
    print('wandb.login() failed — set WANDB_API_KEY or run `!wandb login` interactively.', e)

wandb.login() successful (or WANDB_API_KEY used).


In [5]:
# !wandb login --relogin

## 3) Step 1 — Train a model and log it to W&B as an artifact

This cell trains a small RandomForest on the Iris dataset, saves `model.pkl`, and logs it to W&B in project `classroom-deploy` with artifact name `iris-rf`.

**Run this cell**.

In [7]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import joblib
import wandb

print('Training model...')

run = wandb.init(project='classroom-deploy', job_type='train')
X, y = load_iris(return_X_y=True)
clf = RandomForestClassifier(n_estimators=50, random_state=42)
clf.fit(X, y)
acc = clf.score(X, y)
print(f'Training accuracy: {acc:.4f}')

model_path = 'model.pkl'
joblib.dump(clf, model_path)

artifact = wandb.Artifact(name='iris-rf', type='model', metadata={'accuracy': acc})
artifact.add_file(model_path)
run.log_artifact(artifact)
print('Logged artifact: iris-rf')
run.finish()

Training model...


Training accuracy: 1.0000
Logged artifact: iris-rf


### 3.1 Verify your artifact

- Open the W&B web UI (https://wandb.ai) → your account → project `classroom-deploy` and you should see the run and the artifact.

- Optional: try to list artifacts via the API. If you encounter permission/enterprise differences, rely on the web UI.


In [11]:
import wandb
api = wandb.Api()
try:
    print('Recent model artifacts (project=classroom-deploy):')
    arts = api.artifacts(type_name='model', name='geetheswar-v-iit-pkd/classroom-deploy/iris-rf')
    for a in arts[:10]:
        try:
            print('-', a.id)
        except Exception:
            print('-', str(a))
except Exception as e:
    print('Could not list artifacts via API. Please check the W&B web UI to inspect artifacts.\n', e)

Recent model artifacts (project=classroom-deploy):
- QXJ0aWZhY3Q6MjE1NzU1MjE5MQ==
- QXJ0aWZhY3Q6MjE1NzUxMDUwMw==


## 4) Step 2 — Create a FastAPI app that downloads the model artifact at startup

Run the following cell to create `app/main.py`, `app/requirements.txt`, and a `Dockerfile`. The app expects an environment variable `WANDER_MODEL_ARTIFACT` with the artifact reference, e.g. `your-username/classroom-deploy/iris-rf:latest`. If you want to bake the model into the image, see the notes below.


In [ ]:
import os
os.makedirs('app', exist_ok=True)

main_py = '''import os
import joblib
import wandb
from fastapi import FastAPI
import numpy as np
import dotenv

dotenv.load_dotenv()

WANDB_API_KEY = os.getenv('WANDB_API_KEY')
print(f'W&B API Key loaded: {WANDB_API_KEY is not None}')

app = FastAPI()
MODEL_ARTIFACT = os.environ.get('WANDER_MODEL_ARTIFACT', 'your-username/classroom-deploy/iris-rf:latest')

# Downloads the artifact and returns a loaded model
def load_model_from_wandb(artifact_ref):
    try:
        wandb.login()
    except Exception:
        pass
    api = wandb.Api()
    artifact = api.artifact(artifact_ref)
    path = artifact.download()
    model_file = os.path.join(path, 'model.pkl')
    return joblib.load(model_file)

@app.on_event('startup')
def startup():
    global model
    model = load_model_from_wandb(MODEL_ARTIFACT)

@app.get('/')
def root():
    return {'status': 'ok', 'model_artifact': MODEL_ARTIFACT}

@app.post('/predict')
def predict(features: list[float]):
    arr = np.array(features).reshape(1, -1)
    pred = model.predict(arr)
    return {'prediction': int(pred[0])}
'''

with open('app/main.py', 'w') as f:
    f.write(main_py)

reqs = '''fastapi
uvicorn[standard]
scikit-learn
joblib
wandb
numpy
'''
with open('app/requirements.txt', 'w') as f:
    f.write(reqs)

dockerfile = '''FROM python:3.10-slim
WORKDIR /app
COPY ./requirements.txt /app/requirements.txt
RUN pip install --no-cache-dir -r /app/requirements.txt
COPY ./main.py /app/main.py
ENV PORT=8080
CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8080"]
'''
with open('app/Dockerfile', 'w') as f:
    f.write(dockerfile)

print('Wrote files to ./app:')
print(' - app/main.py')
print(' - app/requirements.txt')
print(' - app/Dockerfile')


Wrote files to ./app:
 - app/main.py
 - app/requirements.txt
 - app/Dockerfile


### 4.1 Quick test: Download the artifact and run inference locally (no FastAPI server needed)

This lets students verify the model artifact and inference logic inside the notebook. It downloads the artifact using the W&B API and loads `model.pkl` then runs a test prediction.


In [14]:
import joblib
import wandb

artifact_ref = 'geetheswar-v-iit-pkd/classroom-deploy/iris-rf:v1'
print('Attempting to download artifact:', artifact_ref)
try:
    api = wandb.Api()
    artifact = api.artifact(artifact_ref)
    path = artifact.download()
    print('Downloaded to', path)
    model = joblib.load(os.path.join(path, 'model.pkl'))
    import numpy as np
    sample = np.array([5.1, 3.5, 1.4, 0.2]).reshape(1, -1)
    print('Sample prediction:', model.predict(sample).tolist())
except Exception as e:
    print('Could not download/run model locally. Check WANDB_API_KEY and artifact ref.\n', e)

Attempting to download artifact: geetheswar-v-iit-pkd/classroom-deploy/iris-rf:v1


wandb:   1 of 1 files downloaded.  


Downloaded to /home/geetheswar/Programming/Sem7/MLOps_git/class/week8/artifacts/iris-rf:v1
Sample prediction: [0]


### 4.2 Run the FastAPI app locally (non-Docker) — good for quick testing on a development machine



In [32]:
!uv add fastapi "uvicorn[standard]" "python-multipart"

Resolved 128 packages in 1ms
Audited 124 packages in 0.04ms


In [41]:
import subprocess
import os
import signal

log_file = "uvicorn_logs.txt"

# Remove old logs if they exist
if os.path.exists(log_file):
    os.remove(log_file)

# Start Uvicorn in background
process = subprocess.Popen(
    ["uv", "run", "uvicorn", "app.main:app", "--host", "0.0.0.0", "--port", "8080", "--reload"],
    stdout=open(log_file, "a"),
    stderr=subprocess.STDOUT,
)

print(f"Started Uvicorn with PID: {process.pid}")


Started Uvicorn with PID: 24711


In [43]:
!curl -X POST -H "Content-Type: application/json" --data '[5.1,3.5,1.4,0.2]' http://localhost:8080/predict

{"prediction":0}

In [44]:
import signal

os.kill(process.pid, signal.SIGTERM)
print("Uvicorn stopped.")

Uvicorn stopped.


## 5) Dockerize the app — build and run on local Ubuntu lab machines

**Build** (from project root):

```bash
cd app
docker build -t iris-wandb:local .
```

**Run** (pass secrets via env vars):

```bash
docker run -e WANDB_API_KEY="$WANDB_API_KEY" \
  -e WANDER_MODEL_ARTIFACT="your-username/classroom-deploy/iris-rf:latest" \
  -p 8080:8080 iris-wandb:local
```

**Test** as before with curl to `/predict`.

**Baking model into image (optional)**: For faster startup and offline runs, download the artifact locally and `COPY` the `model.pkl` into the image during build. That requires a local download step prior to `docker build`.


## 6) Deploy to Google Cloud Run

Prerequisite: `gcloud` configured and authenticated, billing enabled for the project.

Commands (run locally):

```bash
# set project
gcloud config set project YOUR_GCP_PROJECT

# build and push using Cloud Build
gcloud builds submit --tag gcr.io/$GOOGLE_CLOUD_PROJECT/iris-wandb

# deploy
gcloud run deploy iris-wandb \
  --image gcr.io/$GOOGLE_CLOUD_PROJECT/iris-wandb \
  --platform managed \
  --region us-central1 \
  --allow-unauthenticated \
  --set-env-vars=WANDB_API_KEY=$WANDB_API_KEY,WANDER_MODEL_ARTIFACT=your-username/classroom-deploy/iris-rf:latest
```

After this, Cloud Run will provide a public URL you can call with the same `/predict` POST body.


## 7) Optional: GitHub Actions for CI/CD (auto-deploy on push)

Create `.github/workflows/ci-cd.yml` with the following content (example for Cloud Run deployment):

```yaml
name: CI/CD
on:
  push:
    branches: [ main ]
jobs:
  build-deploy:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: google-github-actions/setup-gcloud@v1
        with:
          project_id: ${{ secrets.GCP_PROJECT }}
          service_account_key: ${{ secrets.GCP_SA_KEY }}
      - name: Build & push
        run: |
          gcloud builds submit --tag gcr.io/$GCP_PROJECT/iris-wandb
      - name: Deploy to Cloud Run
        run: |
          gcloud run deploy iris-wandb \
            --image gcr.io/$GCP_PROJECT/iris-wandb \
            --platform managed \
            --region us-central1 \
            --allow-unauthenticated \
            --set-env-vars=WANDB_API_KEY=${{ secrets.WANDB_API_KEY }},WANDER_MODEL_ARTIFACT=your-username/classroom-deploy/iris-rf:latest
```

Use GitHub Secrets for `GCP_SA_KEY` and `WANDB_API_KEY`.


## 8) Troubleshooting & common issues

- **W&B authentication errors**: Ensure `WANDB_API_KEY` is set or run `wandb login`.
- **Artifact not found**: Check artifact path format: `username/project/artifact-name:version` or `username/project/artifact-name:latest`.
- **Docker permissions**: If `docker build` fails due to permissions, ensure your user can run Docker or use `sudo`.
- **Startup latency**: Downloading the artifact at container startup adds latency — consider baking the model into the image for production.



## Instructor notes & grading rubric

Suggested rubric:
- Model logged as W&B Artifact: 10 pts
- Local FastAPI app runs and predicts (no Docker): 20 pts
- Docker image build successful: 20 pts
- Cloud Run (or cloud) deployment functional: 30 pts
- README / Documentation clarity: 20 pts

Instructor tip: Pre-create a W&B project and share artifact refs to reduce setup noise in class.
